## Export Blue Carbon tiffs

* #### **Step 1:** Take Blue carbon Asset from GEE and export to the cloud storage as tiffs
* #### **Step 2:** Take exported tiffs and unify into one single file and reupload to GCS

### Step 1: Export Asset (GEE) to tiff (GCS)

In [2]:
import ee

In [3]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWiVYOUggDLJ-ct3ozWTee8nnp_gAmGF3ic0x0G4lwKNFZf8PoXUqgg



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [6]:
region = ee.Geometry.Polygon([-180, 40, 0,40,180, 40,180,-40,0,-40,-180,-40])
coll = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_total_co2e_1996--2016").select('total_co2e')
collectionList = coll.toList(coll.size())
collectionSize = collectionList.size().getInfo()

In [7]:
i= 0
img = ee.Image(collectionList.get(i))
img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
year = img_name[-4:]

params = {
'image':img,
'description':'toc_'+year,
'bucket':'mangrove_atlas',
'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
'fileFormat':'GeoTIFF',
'region': region.getInfo()['coordinates'], 
'skipEmptyTiles': False,
'maxPixels': 1e12,
'scale':30
}

task = ee.batch.Export.image.toCloudStorage(**params)
task.start()

In [21]:
for i in range(1,collectionSize):
    img = ee.Image(collectionList.get(i))
    img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
    year = img_name[-4:]

    params = {
    'image':img,
    'description':'toc_'+year,
    'bucket':'mangrove_atlas',
    'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
    'fileFormat':'GeoTIFF',
    'region': region.getInfo()['coordinates'], 
    'skipEmptyTiles': False,
    'maxPixels': 1e12,
    'scale': 30,
    }

    task = ee.batch.Export.image.toCloudStorage(**params)
    task.start()

In [13]:
# Check task status
!earthengine task list --status RUNNING

In [38]:
!earthengine task list

GS5LHZ7DHQIZIGWKWITEGJPY  Export.image  toc_2016  COMPLETED  ---
ZSRTQT4UBTJT4JQK65KRQWB2  Export.image  toc_2015  COMPLETED  ---
LPNGZWJSCK36JZAFY342U52L  Export.image  toc_2010  COMPLETED  ---
7TXT6XFST545LSE4NMM6MPJ2  Export.image  toc_2009  COMPLETED  ---
NMDRUGZWALRJCJSKM3SAGTJ5  Export.image  toc_2008  COMPLETED  ---
OW67FABLD4OORUTLQ2UBIRMX  Export.image  toc_2007  COMPLETED  ---
RATJ2X5XOBXHFKWN2ZOTPBOP  Export.image  toc_2000  COMPLETED  ---
NFRDZAQY75G4HNBM73O3TSTW  Export.image  toc_1996  COMPLETED  ---
VE4FBE247HDC56TBMVZHHRJZ  Export.image  toc_1996  FAILED     Unable to write to bucket magrove_atlas (not found).
2WWOJMVYLIQFV6G7OYQ5TY5M  Export.image  toc_1996  FAILED     Export too large: specified 1036802880000 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.
KF7ISWKSLQM7QANWM2KE4FNN  Export.image  toc_2016  FAILED     Internal error.
FL4NEFRGO3KFYVRG45DBBW2Y  Export.image  toc_2015  FAILED     Execution failed; out of memory

### Step 2: Download tiffs (GCS), Unify into 1 file (gdal) and upload (GCS)

#### Download tiffs

## Unify and upload

In [34]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client.from_service_account_json('../../datasets/service_account.json')
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
#https://stackoverflow.com/questions/49748910/python-download-entire-directory-from-google-cloud-storage    
def download_blob(bucket_name, prefix, dl_dir):
    """Downloads a file to the bucket."""
     # bucket_name = 'your-bucket-name'
     # prefix = 'your-bucket-directory/'
     # dl_dir = 'your-local-directory/'

    storage_client = storage.Client.from_service_account_json('../../datasets/service_account.json')
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)  # Get list of files

    for blob in blobs:
        if blob.name.endswith("/"):
                continue
        file_split = blob.name.split("/")
        directory = "/".join(file_split[0:-1])
        Path(directory).mkdir(parents=True, exist_ok=True)
        blob.download_to_filename(blob.name)

    print(
        "Folder {} downloaded to {}.".format(
            prefix, dl_dir
        )
    )
    

In [31]:
y=1996
bucket_name = 'mangrove_atlas'
prefix = f'toc_{y}/'
storage_client = storage.Client.from_service_account_json('../../datasets/service_account.json')
bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files

In [ ]:
download_blob('mangrove_atlas',f'ee_export_tiffs/total_organic_carbon/toc_{y}/toc_{y}',path_in)

In [46]:
y = 1996
bucket_name = 'mangrove_atlas'
#folder='/projects/bigquery/download/shakespeare/'
folder=f'ee_export_tiffs/total_organic_carbon/toc_{y}/'
delimiter='/'
file = f'/toc_{y}'

# Retrieve all blobs with a prefix matching the file.
bucket=storage_client.get_bucket(bucket_name)
# List blobs iterate in folder 
blobs=bucket.list_blobs(prefix=file, delimiter=delimiter) # Excluding folder inside bucket
for blob in blobs:
    print(blob.name)
    #destination_uri = '{}/{}'.format(folder, blob.name) 
    #blob.download_to_filename(destination_uri)

In [ ]:
# [download multiple files]
y=1996
bucket_name = 'mangrove_atlas'
# The "folder" where the files you want to download are
folder=f'ee_export_tiffs/total_organic_carbon/toc_{y}/'


# Retrieve all blobs with a prefix matching the folder
bucket=storage_client.get_bucket(bucket_name)
print(bucket)
blobs=list(bucket.list_blobs(prefix=folder))

for blob in blobs:
    try: 
        if(not blob.name.endswith("/")):
            file_split = blob.name.split("/")
            blob.download_to_filename(f"../../datasets/toc_temporary/{file_split[-1]}")
    except OSError as err:
        print("OS error: {0}".format(err))
        print(file_split[-1])
        continue

# [End download to multiple files]

<Bucket: mangrove_atlas>


In [54]:
file_split = blob.name.split("/")
directory = "/".join(file_split[0:-1])
directory

'ee_export_tiffs/total_organic_carbon/toc_1996'

In [56]:
file_split[-1]

'toc_19960000000000-0000000000.tif'

In [36]:
# Set working directories
import os
path_in = "../../datasets/toc_temporary/"
path_out ="../../datasets/gmw_blue_carbon/"
os.makedirs(path_out, exist_ok=True)
os.makedirs(path_in, exist_ok=True)

In [20]:
%%time
#for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]:
for y in [1996]:
    download_blob('mangrove_atlas',f'ee_export_tiffs/total_organic_carbon/toc_{y}/toc_{y}',path_in)
    print("\nBuilding VRT\n")
    !gdalbuildvrt {path_out}/gmw_toc_{y}.vrt {path_in}/*.tif
    print("\nBuilding GeoTIFF\n")
    !gdal_translate -co "BLOCKXSIZE=512" -co "BLOCKYSIZE=512" -co "TILED=YES" -co "BIGTIFF=YES" -co "COMPRESS=DEFLATE" {path_out}/gmw_toc_{y}.vrt {path_out}/gmw_toc_{y}.tif
    print("\nCopying files to GCS\n")
    upload_blob('mangrove_atlas', f'{path_out}/gmw_toc_{y}.tif', f'ee_export_tiffs/total_organic_carbon/v2/toc_{y}.tif')

FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/toc_temporary/ee_export_tiffs/total_organic_carbon/toc_1996/toc_19960000000000-0000000000.tif'